In [3]:
import pandas as pd
import numpy as np
from datetime import datetime
from dateutil import rrule
import requests
from io import StringIO

In [4]:
# Call Environment Canada API
# Returns a dataframe of data
def getHourlyData(stationID, year, month):
    base_url = "http://climate.weather.gc.ca/climate_data/bulk_data_e.html?"
    query_url = "format=csv&stationID={}&Year={}&Month={}&timeframe=1".format(stationID, year, month)
    api_endpoint = base_url + query_url
    return pd.read_csv(api_endpoint, skiprows=0)

In [5]:
from bs4 import BeautifulSoup  
# Specify Parameters
province = "ON"      # Which province to parse?
start_year = "2006"  # I want the results to go back to at least 2006 or earlier
max_pages = 15        # Number of maximum pages to parse, EC's limit is 100 rows per page, there are about 500 stations in BC with data going back to 2006

# Store each page in a list and parse them later
soup_frames = []

for i in range(max_pages):
    startRow = 1 + i*100
    print('Downloading Page: ', i)
    
    base_url = "http://climate.weather.gc.ca/historical_data/search_historic_data_stations_e.html?"
    queryProvince = "searchType=stnProv&timeframe=1&lstProvince={}&optLimit=yearRange&".format(province)
    queryYear = "StartYear={}&EndYear=2017&Year=2017&Month=5&Day=29&selRowPerPage=100&txtCentralLatMin=0&txtCentralLatSec=0&txtCentralLongMin=0&txtCentralLongSec=0&".format(start_year)
    queryStartRow = "startRow={}".format(startRow)

    response = requests.get(base_url + queryProvince + queryYear + queryStartRow) # Using requests to read the HTML source
    soup = BeautifulSoup(response.text, 'html.parser') # Parse with Beautiful Soup
    soup_frames.append(soup)

In [6]:
import re
# Empty list to store the station data
station_data = []

for soup in soup_frames: # For each soup
    forms = soup.findAll("form", {"id" : re.compile('stnRequest*')}) # We find the forms with the stnRequest* ID using regex 
    for form in forms:
        try:
            # The stationID is a child of the form
            station = form.find("input", {"name" : "StationID"})['value']
            
            # The station name is a sibling of the input element named lstProvince
            name = form.find("input", {"name" : "lstProvince"}).find_next_siblings("div")[0].text
            
            # The intervals are listed as children in a 'select' tag named timeframe
            timeframes = form.find("select", {"name" : "timeframe"}).findChildren()
            intervals =[t.text for t in timeframes]
            
            # We can find the min and max year of this station using the first and last child
            years = form.find("select", {"name" : "Year"}).findChildren()            
            min_year = years[0].text
            max_year = years[-1].text
            
            # Store the data in an array
            data = [station, name, intervals, min_year, max_year]
            station_data.append(data)
        except:
            pass

# Create a pandas dataframe using the collected data and give it the appropriate column names
stations_df = pd.DataFrame(station_data, columns=['StationID', 'Name', 'Intervals', 'Year Start', 'Year End'])
stations_df

,StationID,Name,Intervals,Year Start,Year End
0,26775,ALBERTON,"[Daily, Monthly]",1994,2008
1,42003,ALEXANDRIA,"[Hourly, Daily]",2003,2006
2,42006,ALFRED,"[Hourly, Daily]",2003,2007
3,42967,ALGONQUIN PARK EAST GATE,"[Hourly, Daily, Monthly]",2004,2025
4,4402,ALLISTON NELSON,"[Daily, Monthly]",1973,2008
...,...,...,...,...,...
406,52838,WINDSOR A,"[Hourly, Daily]",2014,2018
407,4715,WINDSOR RIVERSIDE,"[Daily, Monthly]",1866,2024
408,4835,WOODSTOCK,"[Daily, Monthly]",1870,2025
409,6911,WOOLER,"[Daily, Monthly]",1991,2012


In [7]:
#pd.set_option('display.max_columns', None)
#pd.set_option('display.max_rows', None)
monthly_stations = stations_df.loc[stations_df['Intervals'].map(lambda x: 'Monthly' in x)]
monthly_stations

,StationID,Name,Intervals,Year Start,Year End
0,26775,ALBERTON,"[Daily, Monthly]",1994,2008
3,42967,ALGONQUIN PARK EAST GATE,"[Hourly, Daily, Monthly]",2004,2025
4,4402,ALLISTON NELSON,"[Daily, Monthly]",1973,2008
5,4607,AMHERSTBURG,"[Daily, Monthly]",1988,2025
6,6901,APPLETON,"[Daily, Monthly]",1992,2025
...,...,...,...,...,...
405,4716,WINDSOR A,"[Hourly, Daily, Monthly]",1953,2014
407,4715,WINDSOR RIVERSIDE,"[Daily, Monthly]",1866,2024
408,4835,WOODSTOCK,"[Daily, Monthly]",1870,2025
409,6911,WOOLER,"[Daily, Monthly]",1991,2012


In [44]:
start_y = monthly_stations[(monthly_stations['Year Start']=='2002') & (monthly_stations['Year End']=='2025')]
start_y

,StationID,Name,Intervals,Year Start,Year End
263,32128,PARRY SOUND CCG,"[Hourly, Daily, Monthly]",2002,2025
369,31688,TORONTO CITY,"[Hourly, Daily, Monthly]",2002,2025
385,31367,VINELAND STATION RCS,"[Hourly, Daily, Monthly]",2002,2025


,Station Name,Date/Time (LST),Year,Month,Day,Time (LST),Temp (°C)
0,TORONTO CITY,2002-05-01 00:00,2002,5,1,00:00,NaN
1,TORONTO CITY,2002-05-01 01:00,2002,5,1,01:00,NaN
2,TORONTO CITY,2002-05-01 02:00,2002,5,1,02:00,NaN
3,TORONTO CITY,2002-05-01 03:00,2002,5,1,03:00,NaN
4,TORONTO CITY,2002-05-01 04:00,2002,5,1,04:00,NaN
5,TORONTO CITY,2002-05-01 05:00,2002,5,1,05:00,NaN
6,TORONTO CITY,2002-05-01 06:00,2002,5,1,06:00,NaN
7,TORONTO CITY,2002-05-01 07:00,2002,5,1,07:00,NaN
8,TORONTO CITY,2002-05-01 08:00,2002,5,1,08:00,NaN
9,TORONTO CITY,2002-05-01 09:00,2002,5,1,09:00,NaN


In [45]:
#for sID in monthly_stations['StationID']:
station_ID,station_name, empty_rows = [], [], []

for sID in start_y['StationID']:
    stationID = sID
    start_date = datetime.strptime('May2002', '%b%Y')
    end_date = datetime.strptime('Dec2002', '%b%Y')

    frames = []
    for dt in rrule.rrule(rrule.MONTHLY, dtstart=start_date, until=end_date):
        df = getHourlyData(stationID, dt.year, dt.month)
        frames.append(df)

    wt = pd.concat(frames)
    items = ['Station Name','Date/Time (LST)','Year','Month','Day','Time (LST)','Temp (°C)']
    weather = wt[items]
    station_ID.append(stationID)
    station_name.append(weather['Station Name'].iloc[1])
    empty_rows.append(round(((weather['Temp (°C)'].isna().sum())/len(weather['Temp (°C)']))*100,2))

summ =  {"Station ID":station_ID, "Name":station_name,"Empty": empty_rows}
summary = pd.DataFrame(summ)
summary

,Station ID,Name,Empty
0,32128,PARRY SOUND CCG,63.50
1,31688,TORONTO CITY,14.40
2,31367,VINELAND STATION RCS,4.63


In [66]:
# Get Whistler weather data for November 2016 to November 2017
stationID = 31367
start_date = datetime.strptime('Jan2015', '%b%Y')
end_date = datetime.strptime('Dec2024', '%b%Y')

frames = []
for dt in rrule.rrule(rrule.MONTHLY, dtstart=start_date, until=end_date):
    df = getHourlyData(stationID, dt.year, dt.month)
    frames.append(df)

wt2 = pd.concat(frames)
#whistler['Date/Time'] = pd.to_datetime(whistler['Date/Time (LST)'])
#whistler['Temp (°C)'] = pd.to_numeric(whistler['Temp (°C)'])
items2 = ['Station Name','Date/Time (LST)','Year','Month','Day','Time (LST)','Temp (°C)']
weather2 = wt2[items2]
weather2.head(10)

,Station Name,Date/Time (LST),Year,Month,Day,Time (LST),Temp (°C)
0,VINELAND STATION RCS,2015-01-01 00:00,2015,1,1,00:00,-6.5
1,VINELAND STATION RCS,2015-01-01 01:00,2015,1,1,01:00,-6.6
2,VINELAND STATION RCS,2015-01-01 02:00,2015,1,1,02:00,-6.6
3,VINELAND STATION RCS,2015-01-01 03:00,2015,1,1,03:00,-6.5
4,VINELAND STATION RCS,2015-01-01 04:00,2015,1,1,04:00,-6.1
5,VINELAND STATION RCS,2015-01-01 05:00,2015,1,1,05:00,-5.4
6,VINELAND STATION RCS,2015-01-01 06:00,2015,1,1,06:00,-5.3
7,VINELAND STATION RCS,2015-01-01 07:00,2015,1,1,07:00,-4.7
8,VINELAND STATION RCS,2015-01-01 08:00,2015,1,1,08:00,-4.2
9,VINELAND STATION RCS,2015-01-01 09:00,2015,1,1,09:00,-3.0


In [67]:
print(len(weather2))
print(weather2.isna().sum())

87672
Station Name         0
Date/Time (LST)      0
Year                 0
Month                0
Day                  0
Time (LST)           0
Temp (°C)          876
dtype: int64


In [68]:
weather2.to_csv("data4.csv")